<a href="https://colab.research.google.com/github/jon-cb/biopython/blob/master/ovarian_cancer_target_ranking_example_top_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
#positives for ovarian and CAR-T
#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10233107/



In [ ]:
%cd /content/drive/Shareddrives/3-Computing/Crank/v3.0

/content/drive/Shareddrives/3-Computing/Crank/v3.0


# Load Data

In [ ]:
import pandas as pd
import os
import numpy as np
from collections import defaultdict
data_dir="./downloaded/"
data_dir2="./manually_created/"

In [ ]:
#Load human protein atlas data
df_hpa=pd.read_csv(os.path.join(data_dir,"proteinatlas_dec_2022.tsv"),delimiter = "\t")
df_hpa = df_hpa.fillna(0)
df_hpa.rename(columns={"Ensembl": "Gene","Gene": 	"Gene name"}, inplace=True)

In [ ]:
#number of transmembrane regions
df_num_tm=pd.read_csv(os.path.join(data_dir,"AFTM_numbers.txt"), sep="\t")
fixed_cols= [i.strip() for i in df_num_tm.columns]
df_num_tm.columns = fixed_cols
df_num_tm=df_num_tm.rename(columns={"GeneName":"Gene name"})

In [ ]:
#manually generated mapping file to different cancer type IDs and some targets with pre-clincal CARs
df_cancers = pd.read_csv(os.path.join(data_dir2,"cancer_names_latest.csv"))


In [ ]:
#Cell model passport IDs and metadata
df_cmp = pd.read_csv(os.path.join(data_dir,"model_list_latest.csv.gz")) #cell model passports models

In [ ]:
#load proteomics and RNASEQ cell line data from cell model assports
df_prot = pd.read_csv(os.path.join(data_dir,"proteomics_latest.csv.gz"))
df_rna = pd.read_csv(os.path.join(data_dir,"rnaseq_latest.csv.gz"),usecols = ['symbol','tpm',"model_name"])

In [ ]:
if False:
    %cd downloaded
    !wget https://cellbrowser.kaist.ac.kr/ecosystem/ecosystem/exprMatrix.tsv.gz
    !wget https://cellbrowser.kaist.ac.kr/ecosystem/ecosystem/meta.tsv
    %cd ../

In [ ]:
!ls -lSrh downloaded

total 7.3G
drwx------ 2 root root 4.0K Feb 27 17:00 gepia2
-rw------- 1 root root  83K Feb 27 17:00 uniprot_pm.tsv
-rw------- 1 root root 142K Feb 27 17:00 model_list_latest.csv.gz
-rw------- 1 root root 204K Feb 27 17:00 SURFACE_subcell_human_LA_ProtT5.csv
-rw------- 1 root root 216K Mar 12 09:08 AFTM_numbers.txt
-rw------- 1 root root 847K Mar 12 09:01 AFTM.table.txt
-rw------- 1 root root 2.4M Feb 27 17:00 table_S3_surfaceome.csv
-rw------- 1 root root  44M Feb 27 17:00 proteinatlas_dec_2022.tsv
-rw------- 1 root root  70M Feb 27 17:00 proteomics_latest.csv.gz
-rw------- 1 root root 144M May  7 16:21 meta.tsv
-rw------- 1 root root 355M Feb 27 17:00 CRISPR_gene_dependency.csv
-rw------- 1 root root 957M Feb 27 17:01 rnaseq_latest.csv.gz
-rw------- 1 root root 1.8G May  7 16:21 exprMatrix.tsv.gz
-rw------- 1 root root 2.0G Mar 31  2022 CosmicCompleteGeneExpression.tsv.gz
-rw------- 1 root root 2.1G Apr 15 19:49 CosmicCompleteGeneExpression.surfaceome.csv


In [ ]:
#single cell meta data
df_meta = pd.read_csv("downloaded/meta.tsv", sep="\t")

In [ ]:
df_ov=pd.read_csv("manually_created/ovarian_derisked.csv")

In [ ]:
#df_meta[df_meta.cellId.isin(keep_cancer_cells)][["Patient","cellId"]].groupby("Patient").count()

In [ ]:
#df_meta[df_meta["Cancer type"]=="Ovary Normal"].groupby("Cell type").count()

In [ ]:
keep_normal_cells=set(df_meta[(df_meta["Cancer type"]=="Ovary Normal") & (df_meta["Cell type"]=="Epithelial")].cellId)

In [ ]:
keep_cancer_cells=set(df_meta[(df_meta["Cancer type"]=="OV") & (df_meta["Cell type"]=="Epithelial")].cellId)

In [ ]:
cell_patient={}
df_meta_sub=df_meta[df_meta.cellId.isin(keep_cancer_cells | keep_normal_cells)][["cellId","Patient"]].drop_duplicates()
for c,row in df_meta_sub.iterrows():
    cell_patient[row["cellId"]] = row["Patient"]
print(len(set(cell_patient.values())))

56


In [ ]:
#Load different annotations of plasma membrane protein ids
df_pm = pd.read_csv(os.path.join(data_dir,"table_S3_surfaceome.csv"))
df_pm.rename(columns={"UniProt gene": "Gene name"}, inplace=True)
df_cmt5=pd.read_csv(os.path.join(data_dir,"SURFACE_subcell_human_LA_ProtT5.csv"),usecols=[0,2], header=None)
df_cmt5.columns = ["uacc", "nres"]
df_pm_uniprot = pd.read_csv(os.path.join(data_dir,"uniprot_pm.tsv"), delimiter="\t")

In [ ]:

#subset HPA so its only got surfaceome proteins
df_hpa_sub =df_hpa[ (df_hpa["Gene name"].isin(set(df_ov.gene))) | ( df_hpa["Uniprot"].isin(set(df_cmt5.uacc))  ) |  (df_hpa["Gene name"].isin(set(df_pm["Gene name"])))  |   (df_hpa["Uniprot"].isin(set(df_pm_uniprot["Entry"])))   ]

In [ ]:
keep_genes = set(df_hpa_sub["Gene name"])

In [ ]:
import gzip
import csv
keep_cols=[]
col_cell={}
res=defaultdict(list)

for c, line in enumerate(csv.reader(gzip.open("downloaded/exprMatrix.tsv.gz","rt"), delimiter="\t")):

    #if c > 20: break
    if c ==0:
      for c2, cell_id in enumerate(line):
        if cell_id in (keep_cancer_cells) or cell_id in keep_normal_cells:
          keep_cols.append(c2)
          col_cell[c2]=cell_id

    else:


      gene = line[0]
      if gene not in keep_genes: continue
      for col in keep_cols:
        cell_id = col_cell[col]
        patient=cell_patient[cell_id]
        exp = float(line[col])
        if exp <0: continue
        res["cell_id"].append(cell_id)
        res["disease"].append(cell_id in keep_cancer_cells)
        res["exp"].append(exp)
        res["patient"].append(patient)
        res["gene"].append(gene)

      print(c,gene)
df_res = pd.DataFrame(res)
df_res.to_csv("OV/single_cell.csv")

In [ ]:
df_res = pd.read_csv("OV/single_cell.csv")

In [ ]:
df_res

,Unnamed: 0,cell_id,disease,exp,patient,gene
0,0,ov_GSE184880_GSM5599225_ACCTACCGTCTGTGCG-1,True,0.526932,ov_GSE184880_GSM5599225,TNFRSF18
1,1,ov_GSE184880_GSM5599225_AGAGCAGTCCGACAGC-1,True,2.343564,ov_GSE184880_GSM5599225,TNFRSF18
2,2,ov_GSE184880_GSM5599225_AGTAACCGTCGCTTAA-1,True,1.610549,ov_GSE184880_GSM5599225,TNFRSF18
3,3,ov_GSE184880_GSM5599225_ATGAAAGAGTGCACTT-1,True,3.794946,ov_GSE184880_GSM5599225,TNFRSF18
4,4,ov_GSE184880_GSM5599225_CAACAGTGTTTACTTC-1,True,2.161046,ov_GSE184880_GSM5599225,TNFRSF18
...,...,...,...,...,...,...
2061721,2061721,ovary_GSE118127_GSM3319036_TTTACTGAGAAACGCC-1,False,0.072565,ovary_GSE118127_P7,MT-CO1
2061722,2061722,ovary_GSE118127_GSM3319036_TTTACTGGTATAAACG-1,False,0.188210,ovary_GSE118127_P7,MT-CO1
2061723,2061723,ovary_GSE118127_GSM3319036_TTTCCTCCAAATCCGT-1,False,0.949411,ovary_GSE118127_P7,MT-CO1
2061724,2061724,ovary_GSE118127_GSM3319036_TTTCCTCTCACCGGGT-1,False,0.612409,ovary_GSE118127_P7,MT-CO1


In [ ]:
df_res2 = df_res.groupby(["patient","disease","gene"]).sum().reset_index()

In [ ]:
df_res3 = df_res.groupby(["patient","disease"]).nunique().reset_index()[["patient","disease","cell_id"]].rename(columns={"cell_id":'num_cells'})

In [ ]:
df_res_fin = df_res2.merge(df_res3, on=["patient", "disease"])

In [ ]:
df_res_fin["mean_exp"] = df_res_fin["exp"] / df_res_fin["num_cells"]

In [ ]:
df_norm = df_res_fin[(df_res_fin.disease==False) ][["gene","mean_exp"]].groupby(["gene"]).median().reset_index()
df_dis = df_res_fin[(df_res_fin.disease==True) ][["gene","mean_exp"]].groupby(["gene"]).median().reset_index()
df_res_fin = df_dis.merge(df_norm, on="gene", how="left").fillna(0).rename(columns={"mean_exp_x":"median_sc_cancer","mean_exp_y":"median_sc_norm"})
df_res_fin["diff"] = df_res_fin["median_sc_cancer"] - df_res_fin["median_sc_norm"]

In [ ]:
df_sc = df_res_fin

In [ ]:
df_sc

,gene,median_sc_cancer,median_sc_norm,diff
0,ABCA1,0.126347,0.094686,0.031661
1,ABCA10,0.024814,0.333360,-0.308546
2,ABCA6,0.014159,0.290688,-0.276529
3,ABCA8,0.009520,0.530597,-0.521078
4,ABCA9,0.019248,0.404483,-0.385235
...,...,...,...,...
1046,XKR4,0.095651,0.336700,-0.241050
1047,XKR6,0.122329,0.101110,0.021219
1048,XPNPEP2,0.009592,0.058356,-0.048764
1049,ZAP70,0.014610,0.063986,-0.049376


# CB

# load CDM

In [ ]:
#load cancer dependency map data
df_cdm = pd.read_csv(os.path.join(data_dir,"CRISPR_gene_dependency.csv"))
df_cdm=df_cdm.set_index("DepMap_ID")

# Parse useful scores out of human protein atlas

In [ ]:
#Cells we are happy to tolerate expression in, can be different based on cancer type
cells_tolerate=["sertoli cells", "trophoblasts","sperm","leydig","peritubular","Hofbauer cells","Theca cells","Thymic",
                #"Plasma cells", "B-cells"
                ]

In [ ]:
df_cancers

In [ ]:

#PARSE HPA SINGLE CELL EXPRESSION DATA

res=defaultdict(list)
for index, row in df_hpa_sub.iterrows():
    tcell_scores=[0] #EXPRESSION IN TCELLS IS VERY BAD FOR CAR-T

    exp_tolerate =[] #SOME HUMAN CELLS WE ARE HAPPY TO HAVE EXPRESSION IN

    exp_not_tolerate =[] #SOME CELLS WE ARE NOT HAPPY TO HAVE EXPRESSION IN

    for col in df_hpa_sub.columns:

      if col.startswith("Single Cell Type RNA - "):# FOR SINGLE CELL EXPRESSION DATA

        #TPM EXPRESSION VALUE
        exp_tpm = float(row[col])

        #RECORD IF EXPRESSED IN T-CELLS, THIS IS BAD
        if col.find(" T-cells ") > -1: tcell_scores.append(exp_tpm)

        #CELL TYPE
        cell = col.replace("Single Cell Type RNA - ","").split("[")[0].strip().lower()

        #RECORD EXPRESSION AND IF ITS IN AN OK CELL OR A BAD CELL TO BE EXPRESSED IN
        tolerate =False
        for i in cells_tolerate:
            if cell.lower().find(i.lower()) > -1:
                tolerate=True

        if tolerate: exp_tolerate.append(exp_tpm)
        else:exp_not_tolerate.append(exp_tpm)

      #RECORD IF EXPRESSED IN T-CELLS, THIS IS BAD
      if col.find("Blood RNA - naive") > -1 and col.lower().find("t-cell") > -1:
        exp_tpm = float(row[col])
        tcell_scores.append(exp_tpm)



    res["t_cell"].append(np.max(tcell_scores))
    res["Gene name"].append(row["Gene name"])
    res["exp_tolerate"].append(np.sum(exp_tolerate))
    res["exp_bad"].append(np.sum(exp_not_tolerate))

df_hpa_res = pd.DataFrame(res)

In [ ]:
df_rna_median=df_rna[["symbol","tpm"]].groupby("symbol").median()

# combine datasets to rank genes

In [ ]:
df_gepia_deg = pd.read_csv(os.path.join(data_dir,"gepia2", "ov_degs.txt"),delimiter="\t")
df_gepia_deg.rename(columns={"Gene Symbol": "Gene name","Log2(Fold Change)": "TCGA_fc","Median (Tumor)": "TCGA_tpm"},inplace=True)


In [ ]:
#keep_cancs == cancers we want to generate rankings for see the associated csv file  'cancer_names_latest.csv' for more info on cancer codes
keep_cancs =["OV"]
df_results = []
df_pan_cancer=pd.DataFrame()

for c, row in df_cancers.dropna(subset="cb_cancer_id_lev1").iterrows():
    cancer_name = row.cb_cancer_id_lev1

    if cancer_name not in keep_cancs: continue
    print("CANCER",cancer_name)


    #Get BROAD IDS and cell model passport IDs that correspond to our cancer of interest
    df_cmp_sub = df_cmp[(df_cmp.cancer_type==row["CMP"]) | (df_cmp.cancer_type_detail==row["CMP"]) ][["model_id" ,"model_name","cancer_type", "BROAD_ID"]]

    cancer_cmps = set(df_cmp_sub.model_name)
    dmap_ids = set(df_cmp_sub.BROAD_ID)

    #Get logFC of median values for cell line RNASEq data for our cancer cell lines of interest compared to all cell lines
    df_rna_sub = df_rna[df_rna.model_name.isin(cancer_cmps)]
    df_rna_sub_median = df_rna_sub[["symbol","tpm"]].groupby("symbol").median()
    df_rna_sub_score = df_rna_sub_median.reset_index().merge(df_rna_median.reset_index(), on="symbol")
    df_rna_sub_score["log_fc"] = np.log((df_rna_sub_score["tpm_x"] + 1.0) /  (df_rna_sub_score["tpm_y"] + 1.0))


    #Get the median proteomics level for all genes in our cell-lines from the current cancer
    df_prot_sub = df_prot[df_prot.model_name.isin(cancer_cmps)]
    df_prot_sub_score=df_prot_sub[["symbol", "zscore"]].groupby("symbol").median().reset_index()


    #Get some measure of how easy is it to loose this gene through evolutionary process through cancer depepndency map data

    df_cdm_canc = df_cdm[df_cdm.index.isin(dmap_ids)].mean()
    df_cdm_other = df_cdm[~df_cdm.index.isin(dmap_ids)].mean()
    df_cdm_comb=pd.concat([df_cdm_canc, df_cdm_other],axis=1)
    df_cdm_comb.columns =["cdm_score","cdm_score_other"]
    gids=[]
    for g in df_cdm_comb.index:gids.append(g.split()[0])
    df_cdm_comb["Gene name"] = gids


    df_final = df_hpa_res.merge(df_cdm_comb[["Gene name","cdm_score"]], on="Gene name", how="left")

    df_final = df_final.merge(df_gepia_deg,on="Gene name", how="left")
    df_final = df_final.merge(df_rna_sub_score.rename(columns={"symbol": "Gene name","log_fc": "rna_log_fc"})[["Gene name","rna_log_fc"]], on="Gene name", how="left")


    df_final=df_final.merge(df_prot_sub_score.rename(columns={"symbol": "Gene name","zscore":"proteomics"})[["Gene name","proteomics"]], on="Gene name", how="left")
    df_final = df_final.fillna(value=0)

    #df_final.to_excel(writer,cancer_name)



CANCER OV


In [ ]:
df_final=df_final.merge(df_ov, left_on="Gene name",right_on ="gene", how="left")
df_final=df_final.merge(df_sc, left_on="Gene name",right_on ="gene", how="left" )

In [ ]:
df_final

# Add extra gene info to the selected genes

In [ ]:
#add how many transmembrane segnemts does it have (1 is better for us)
df_final = df_final.merge(df_num_tm[["Gene name", "#AFTM","#UniProtTM"]], how="left", on="Gene name")

In [ ]:
df_final.to_csv("./OV/ov_scores.csv")

In [ ]:
df_final

,t_cell,Gene name,exp_tolerate,exp_bad,cdm_score,Gene ID,TCGA_tpm,Median (Normal),TCGA_fc,adjp,...,gene_x,clin_trial,aka,source,gene_y,median_sc_cancer,median_sc_norm,diff,#AFTM,#UniProtTM
0,2.3,TSPAN6,2386.5,2919.9,0.028137,0,0.000,0.000,0.000,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0
1,0.1,TNMD,10.8,101.2,0.042408,0,0.000,0.000,0.000,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
2,78.4,FGR,122.6,1142.3,0.019337,0,0.000,0.000,0.000,0.000000e+00,...,NaN,NaN,NaN,NaN,FGR,0.011679,0.03124,-0.019561,NaN,NaN
3,38.9,NIPAL3,86.4,1542.1,0.008713,ENSG00000001461.16,9.725,4.285,1.021,2.270000e-21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0
4,15.3,ENPP4,104.9,1268.8,0.017626,ENSG00000001561.6,4.985,1.605,1.200,9.370000e-28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4969,0.0,OR4F29,0.0,0.0,0.000000,0,0.000,0.000,0.000,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4970,0.0,SMIM41,23.5,8.2,0.000000,0,0.000,0.000,0.000,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
4971,0.0,EEF1AKMT4-ECE2,0.0,0.0,0.000000,0,0.000,0.000,0.000,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
4972,3.4,UPK3BL2,0.5,1.9,0.000000,0,0.000,0.000,0.000,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
